In [23]:
import pandas as pd
import matplotlib.pyplot as plt
import json
import numpy as np


In [24]:
df = pd.read_json("../VLSP2023-LTER-Data/legal_passages.json")

In [25]:
def getTextInArticles(df):
  result = []  # Danh sách lưu kết quả
  # Truy cập các cột và hàng trong DataFrame
  for index, row in df.iterrows():
    item_id = row['id']
    articles = row['articles']
    for article in articles:
        article_id = article['id']
        text = article['text']

        result.append({'article_id': article_id, 'text': text, 'id': item_id})  # Thêm kết quả vào danh sách

  result_df = pd.DataFrame(result)  # Chuyển đổi danh sách thành DataFrame
  # result_df['index'] = range(len(result_df))
  return result_df

In [26]:
# Chạy hàm để lấy DataFrame
new_df = getTextInArticles(df)

# Chuyển DataFrame thành danh sách các từ điển
list_from_df = new_df.to_dict('records')

# Hiện thị danh sách này
print(list_from_df[0])

{'article_id': '1', 'text': 'Phạm vi điều chỉnh\n\nLuật này quy định về viên chức; quyền nghĩa vụ của viên chức; tuyển dụng, sử dụng và quản lý viên chức trong đơn vị sự nghiệp công lập.', 'id': 'Luật Viên chức 2010'}


In [27]:
import json
import yaml
from unidecode import unidecode

In [28]:
yaml_data = {}
utter_list = []  # Danh sách để lưu các giá trị utter
for item in list_from_df:
    utter = f'utter_{unidecode(item["id"].replace(" ", "").replace(",",""))}_{item["article_id"]}'
    utter_list.append(utter)  # Thêm giá trị utter vào danh sách
    yaml_data[utter] = {"- text": item["text"]}


In [10]:
# Tên file mà bạn muốn lưu dữ liệu vào
file_name = 'Data_domain.yml'

# Ghi dữ liệu YAML vào tệp tin
with open(file_name, 'w', encoding='utf-8') as file:
    yaml.dump(yaml_data, file, default_flow_style=False, allow_unicode=True)

print(f"Dữ liệu đã được ghi vào '{file_name}' thành công.")

Dữ liệu đã được ghi vào 'Data_domain.yml' thành công.


In [29]:
train_df = pd.read_json("../VLSP2023-LTER-Data/train.json")

In [30]:
def getNewTrainData(df):
  result = []  # Danh sách lưu kết quả
  # Truy cập các cột và hàng trong DataFrame
  for index, row in df.iterrows():
    question_id = row['example_id']
    label = row['label']
    text = row['statement']
    articles = row['legal_passages']

    for article in articles:
        article_id = article['article_id']
        law_id = article['law_id']

        result.append({'example_id': question_id,'label':label, 'statement': text, 'article_id': article_id, 'law_id': law_id, })  # Thêm kết quả vào danh sách

  result_df = pd.DataFrame(result)  # Chuyển đổi danh sách thành DataFrame
  result_df['index'] = range(len(result_df))
  return result_df

In [31]:
new_relevant_articles = getNewTrainData(train_df)
new_relevant_articles.head()

,example_id,label,statement,article_id,law_id,index
0,q9zjh7Uw7Q,No,Người xem dưới 16 tuổi được xem phim có nội du...,32,Luật Điện ảnh 2022,0
1,ckQFn8y202,No,"Trong vòng 03 ngày làm việc, kể từ ngày người ...",30,"Luật Phòng, chống ma túy 2021",1
2,3ROu621ZEO,Yes,Viên chức có 02 năm liên tiếp bị phân loại đán...,29,Luật Viên chức 2010,2
3,VT1QuVmhCc,Yes,Các biện pháp cai nghiện ma túy là những biện ...,28,"Luật Phòng, chống ma túy 2021",3
4,0MwITLtbmg,No,Viên chức thuộc một đơn vị sự nghiệp công lập ...,14,Luật Viên chức 2010,4


In [32]:
# Chạy hàm để lấy DataFrame
new_relevant_articles = getNewTrainData(train_df)

# Chuyển DataFrame thành danh sách các từ điển
list_from_new_relevant_articles = new_relevant_articles.to_dict('records')

# Hiện thị danh sách này
print(list_from_new_relevant_articles[0])

{'example_id': 'q9zjh7Uw7Q', 'label': 'No', 'statement': 'Người xem dưới 16 tuổi được xem phim có nội dung thuộc phân loại T18', 'article_id': '32', 'law_id': 'Luật Điện ảnh 2022', 'index': 0}


In [33]:
yaml_data = {}
intent_list = []  # Danh sách để lưu các giá trị utter
for item in list_from_new_relevant_articles:
    intent = f'- intent:{unidecode(item["law_id"].replace(" ", "").replace(",",""))}_{item["article_id"]}'
    intent_list.append(intent)  # Thêm giá trị utter vào danh sách
    yaml_data[intent] = {"examples: |": item["statement"]}


In [22]:
# Tên file mà bạn muốn lưu dữ liệu vào
file_name = 'Data_intent.yml'

# Ghi dữ liệu YAML vào tệp tin
with open(file_name, 'w', encoding='utf-8') as file:
    yaml.dump(yaml_data, file, default_flow_style=False, allow_unicode=True)

print(f"Dữ liệu đã được ghi vào '{file_name}' thành công.")

Dữ liệu đã được ghi vào 'Data_intent.yml' thành công.


In [23]:
# Tên file mà bạn muốn lưu danh sách vào
intent_file = 'intent_list.txt'

# Ghi danh sách các giá trị utter vào file text
with open(intent_file, 'w', encoding='utf-8') as file:
    for intent in intent_list:
        file.write(intent + '\n')  # Ghi từng giá trị utter và xuống dòng

print(f"Danh sách các giá trị utter đã được ghi vào '{intent_file}' thành công.")

Danh sách các giá trị utter đã được ghi vào 'intent_list.txt' thành công.


In [24]:
yaml_data = {}
intent_list = []  # Danh sách để lưu các giá trị utter
for item in list_from_new_relevant_articles:
    intent = f'- intent:{item["law_id"].replace(" ", "")}_{item["article_id"]}'
    intent_list.append(intent)  # Thêm giá trị utter vào danh sách
    yaml_data[intent] = {"examples: |": item["statement"]}


In [39]:
output_story = []
for item in list_from_df:
    story = f"{item['id'].replace(' ', '')}_{item['article_id']}"
    steps = [
        {"intent": f"{unidecode(item['id'].replace(' ', '').replace(',',''))}_{item['article_id']}"},
        {"action": f"utter_{unidecode(item['id'].replace(' ', '').replace(',',''))}_{item['article_id']}"}
    ]
    output_story.append({"- story": story, "steps": steps})


In [40]:
# Tên file mà bạn muốn lưu dữ liệu vào
file_name = 'Data_story.yml'

# Ghi dữ liệu YAML vào tệp tin
with open(file_name, 'w', encoding='utf-8') as file:
    json.dump(output_story, file, allow_unicode=True)

print(f"Dữ liệu đã được ghi vào '{file_name}' thành công.")

Dữ liệu đã được ghi vào 'Data_story.yml' thành công.


In [41]:
import yaml

# Cấu trúc dữ liệu output_story trước khi ghi vào file
formatted_output = "\n".join([yaml.dump(d, default_flow_style=False, allow_unicode=True) for d in output_story])

# Tên file mà bạn muốn lưu dữ liệu vào
file_name = 'Data_story.yml'

# Ghi dữ liệu đã định dạng vào tệp tin
with open(file_name, 'w', encoding='utf-8') as file:
    file.write(formatted_output)

print(f"Dữ liệu đã được ghi vào '{file_name}' thành công.")


Dữ liệu đã được ghi vào 'Data_story.yml' thành công.
